機械学習特論 第6回課題 モデル・パラメータの探索

In [1]:
# google colab で実行する場合は、次の行の先頭の # を削除してこのブロックを実行する
#!pip install japanize-matplotlib


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [3]:
# Fashion-MNISTデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST')
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y) 
    return x, y

# 一括処理のためにデータセットの辞書を作成
dataset = {'fashon-mnist': load_fashion_mnist_data()}

In [23]:
# モデル・パラメータ探索のためにモデル・パラメータ探索範囲の辞書を作成
model = {
    # svm
    'SVC': {'model': svm.SVC(),
            'parameters': {'kernel': ["linear", "rbf"],
                           'C': [0.1,1,10],}, 
           },
    # ランダムフォレスト
    'randomforest':{'model': RandomForestClassifier(random_state=1),
                    'parameters': {'max_depth': [5,10,15],
                                   'n_estimators': [50,100,150]}, 
                   },
}


In [24]:
# 辞書に格納したデータセット・モデルについて最良パラメータを GridSearchCV 探索して性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1) # 検証用データに25%を割当て
    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデル・パラメータを探索
        clf = model[model_key]['model']
        parameters = model[model_key]['parameters']
        clf_search = GridSearchCV(clf, parameters)
        clf_search.fit(x_train, np.array(y_train).ravel())

        # 探索したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf_search.best_estimator_.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf_search.best_estimator_.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'model:{model_key} best parameters:', clf_search.best_params_,
              f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')


## dataset:fashon-mnist  x_train:5250 x_test:1750 y_train:5250 y_test:1750
model:SVC best parameters: {'C': 10, 'kernel': 'rbf'} accuracy_score: train_data: 0.9899 test_data: 0.87486
model:randomforest best parameters: {'max_depth': 15, 'n_estimators': 100} accuracy_score: train_data: 0.99771 test_data: 0.844


In [19]:
# モデル・パラメータ探索のためにモデル・パラメータ探索範囲の辞書を作成
from scipy import stats
model = {
    # svm
    'SVC': {'model': svm.SVC(),
            'parameters': {'kernel': ["linear", "rbf"],
                           'C': stats.expon(scale=1),}, 
           },
    # ランダムフォレスト
    'randomforest':{'model': RandomForestClassifier(random_state=1),
                    'parameters': {'max_depth': stats.randint(5,15),
                                   'n_estimators': stats.randint(50,200)}, 
                   },
}


In [20]:
# 辞書に格納したデータセット・モデルについて最良パラメータを RandomizedSearchCV 探索して性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1) # 検証用データに25%を割当て
    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデル・パラメータを探索
        clf = model[model_key]['model']
        parameters = model[model_key]['parameters']
        clf_search = RandomizedSearchCV(clf, parameters)
        clf_search.fit(x_train, np.array(y_train).ravel())

        # 探索したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf_search.best_estimator_.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf_search.best_estimator_.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'model:{model_key} best parameters:', clf_search.best_params_,
              f'model:{model_key} accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')


## dataset:fashon-mnist  x_train:5250 x_test:1750 y_train:5250 y_test:1750
model:SVC best parameters: {'C': 3.866308691595505, 'kernel': 'rbf'} model:SVC accuracy_score: train_data: 0.95981 test_data: 0.87143
model:randomforest best parameters: {'max_depth': 12, 'n_estimators': 160} model:randomforest accuracy_score: train_data: 0.98286 test_data: 0.84629
